In [118]:
import os
import logging
import sys

import openai
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import (
    Document,
    KnowledgeGraphIndex,
    StorageContext,
)
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.llms.openai import OpenAI
from llama_index.graph_stores.nebula import NebulaGraphStore


load_dotenv()

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
)

openai.api_key = os.environ["OPENAI_API_KEY"]

In [119]:
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,llamaindex
1,policies
2,policies_2
3,policies_3
4,policies_aa


In [120]:
%ngql USE policies_aa;
%ngql MATCH ()-[e]->() RETURN e LIMIT 10000;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e
0,"(""100 - 160 wh"")-[:relationship@-8438811214395..."
1,"(""100 - 160 wh"")-[:relationship@13461996532723..."
2,"(""160 - 300 wh"")-[:relationship@-8438811214395..."
3,"(""160 - 300 wh"")-[:relationship@13461996532723..."
4,"(""1st bag fee"")-[:relationship@-84388112143955..."
...,...
776,"(""You"")-[:relationship@7869679751083147176{nam..."
777,"(""You"")-[:relationship@8622192678635854172{nam..."
778,"(""You can travel with"")-[:relationship@-128523..."
779,"(""Your bag"")-[:relationship@-45884617147977354..."


In [121]:
space_name = "policies_aa"
edge_types, rel_prop_names = ["relationship"], [
    "name"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

llm = OpenAI(temperature=0, model="gpt-4-turbo") #  model="gpt-3.5-turbo")

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    llm=llm,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever
)

In [122]:
response = query_engine.query("Tell me about American Airlines?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>American Airlines has various policies and regulations regarding items that can be flown with, including restrictions on certain items like spray cans, butane fuel, propane tanks, CO2 cartridges, self-inflating rafts, and recreational oxygen. They also have policies related to carry-on bags, checked bags, and portable electronic devices containing lithium batteries. Additionally, American Airlines offers complimentary bags and has a policy for restricted items.</b>

In [123]:
response = query_engine.query("Can I travel with pets?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.


<b>Empty Response</b>

In [116]:
query_engine.query("Tell me about American Airlines")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='American Airlines has policies related to various aspects of air travel, such as restricted items, checked bag allowances, and basic economy travel. They also have specific policies for travel to certain regions like Panama and South America, as well as seasonal travel restrictions. Additionally, American Airlines enforces policies regarding delayed bags, gasoline-powered tools, and what items are allowed to be flown with.', source_nodes=[NodeWithScore(node=TextNode(id_='b728a61c-6129-4eac-ba79-abb79651285b', embedding=None, metadata={'kg_rel_map': {'American{name: American}': ['American{name: American} -[relationship:{name: Does not allow}]-> Transport marijuana on flights{name: Transport marijuana on flights}'], 'Airline{name: Airline}': ['Airline{name: Airline} <-[relationship:{name: Applies to}]- Policy{name: Policy} -[relationship:{name: Includes}]-> Optional fees{name: Optional fees}', 'Airline{name: Airline} <-[relationship:{name: Applies to}]- Policy{name: Po

In [47]:
print(query_engine.query("Tell me about Lithium batteries"))

Lithium batteries are a type of battery that is intended for personal use and is commonly found in devices such as laptops. They are also known to be individually protected and are often included in carry-on bags for safety reasons. Additionally, lithium batteries have various types such as lithium-ion, lithium polymer, and lithium iron phosphate.


In [64]:
nodes = graph_rag_retriever.retrieve("Tell me about Lithium batteries")

In [63]:
print(nodes[0].text)

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
Lithium{name: Lithium} <-[relationship:{relationship: Containing}]- Devices{name: Devices} -[relationship:{relationship: Are not intended for}]-> Personal use{name: Personal use}
Lithium{name: Lithium} <-[relationship:{relationship: Contain}]- Cells{name: Cells} -[relationship:{relationship: Are intended for}]-> Personal use{name: Personal use}
Lithium{name: Lithium} <-[relationship:{relationship: Containing}]- Devices{name: Devices} -[relationship:{relationship: Containing}]-> Cells{name: Cells}
Lithium{name: Lithium} -[relationship:{relationship: Is type of}]-> Battery{name: Battery} -[relationship:{relationship: Is}]-> Protected{name: Protected}
Lithium{name: Lithium} -[relationship:{relationship: Is type of}]-> Battery{name: Battery} -[relationship:{relationship: Is}]-> Lith

In [57]:
nodes[0].metadata

{'kg_rel_map': {'Lithium{name: Lithium}': ['Lithium{name: Lithium} <-[relationship:{relationship: Containing}]- Devices{name: Devices} -[relationship:{relationship: Are not intended for}]-> Personal use{name: Personal use}',
   'Lithium{name: Lithium} <-[relationship:{relationship: Contain}]- Cells{name: Cells} -[relationship:{relationship: Are intended for}]-> Personal use{name: Personal use}',
   'Lithium{name: Lithium} <-[relationship:{relationship: Containing}]- Devices{name: Devices} -[relationship:{relationship: Containing}]-> Cells{name: Cells}',
   'Lithium{name: Lithium} -[relationship:{relationship: Is type of}]-> Battery{name: Battery} -[relationship:{relationship: Is}]-> Protected{name: Protected}',
   'Lithium{name: Lithium} -[relationship:{relationship: Is type of}]-> Battery{name: Battery} -[relationship:{relationship: Is}]-> Lithium battery acceptance policy{name: Lithium battery acceptance policy}',
   'Lithium{name: Lithium} <-[relationship:{relationship: Contain}]- C

In [69]:
query = query_engine.query("Are Lithium batteries allowed in my carry-on?")
print(query.response)
print(query.source_nodes)

Yes, Lithium batteries are allowed in your carry-on.
[NodeWithScore(node=TextNode(id_='21ec8a76-3bfd-4355-bd5f-04129b150f43', embedding=None, metadata={'kg_rel_map': {'Lithium-ion batteries{name: Lithium-ion batteries}': ['Lithium-ion batteries{name: Lithium-ion batteries} <-[relationship:{relationship: Allows}]- Airline{name: Airline} -[relationship:{relationship: Pay for}]-> Checked bags online{name: Checked bags online}', 'Lithium-ion batteries{name: Lithium-ion batteries} <-[relationship:{relationship: Allows}]- Airline{name: Airline} -[relationship:{relationship: Issued within/between}]-> U.s. (including hi/ak{name: U.s. (including hi/ak}', 'Lithium-ion batteries{name: Lithium-ion batteries} <-[relationship:{relationship: Allows}]- Airline{name: Airline} -[relationship:{relationship: Allows}]-> Watches for travel{name: Watches for travel}', 'Lithium-ion batteries{name: Lithium-ion batteries} <-[relationship:{relationship: Allows}]- Airline{name: Airline} -[relationship:{relationsh

In [70]:
query = query_engine.query("I'm flying American Airlines. Are Lithium batteries allowed in my carry-on?")
print(query.response)
print(query.source_nodes)

Yes, Lithium batteries are allowed in carry-on baggage when flying American Airlines.
[NodeWithScore(node=TextNode(id_='b6b12fb5-3063-4ee4-8c45-9ccb8b0203cc', embedding=None, metadata={'kg_rel_map': {'American{name: American}': ['American{name: American} <-[relationship:{relationship: Is}]- App{name: App} -[relationship:{relationship: Is}]-> Mobile{name: Mobile}', 'American{name: American} <-[relationship:{relationship: Is}]- App{name: App} -[relationship:{relationship: Choose}]-> Flight{name: Flight}', 'American{name: American} <-[relationship:{relationship: Is}]- App{name: App} <-[relationship:{relationship: Check-in}]- Bags{name: Bags}', "American{name: American} <-[relationship:{relationship: Is}]- App{name: App} <-[relationship:{relationship: Don't have}]- App{name: App}", 'American{name: American} -[relationship:{relationship: Marketed}]-> Itineraries{name: Itineraries} <-[relationship:{relationship: Markets and operates}]- American airlines{name: American airlines}', 'American{n

In [74]:
query.source_nodes[0].dict()

{'node': {'id_': 'b6b12fb5-3063-4ee4-8c45-9ccb8b0203cc',
  'embedding': None,
  'metadata': {'kg_rel_map': {'American{name: American}': ['American{name: American} <-[relationship:{relationship: Is}]- App{name: App} -[relationship:{relationship: Is}]-> Mobile{name: Mobile}',
     'American{name: American} <-[relationship:{relationship: Is}]- App{name: App} -[relationship:{relationship: Choose}]-> Flight{name: Flight}',
     'American{name: American} <-[relationship:{relationship: Is}]- App{name: App} <-[relationship:{relationship: Check-in}]- Bags{name: Bags}',
     "American{name: American} <-[relationship:{relationship: Is}]- App{name: App} <-[relationship:{relationship: Don't have}]- App{name: App}",
     'American{name: American} -[relationship:{relationship: Marketed}]-> Itineraries{name: Itineraries} <-[relationship:{relationship: Markets and operates}]- American airlines{name: American airlines}',
     'American{name: American} -[relationship:{relationship: Marketed}]-> Itinerari

In [83]:
# thanks! https://github.com/run-llama/llama_index/issues/11034
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.legacy.query_engine.knowledge_graph_query_engine import DEFAULT_NEO4J_NL2CYPHER_PROMPT
    
nl2kg_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    verbose=True,
    refresh_schema=True,
    graph_query_synthesis_prompt=DEFAULT_NEO4J_NL2CYPHER_PROMPT,
    llm=llm,
)

In [87]:
question = "Are Lithium batteries allowed in my carry-on?"

response_nl2kg = nl2kg_query_engine.query(question)

# Cypher:
print("The Cypher Query is:")
query_string = nl2kg_query_engine.generate_query(question)
display(
    Markdown(
        f"""
```cypher
{query_string}
```
"""
    )
)

%ngql {query_string}

# Answer:
print("The Answer is:")
display(Markdown(f"<b>{response_nl2kg}</b>"))

Graph Store Query:
MATCH (e1:entity {name: "Lithium batteries"})-[:relationship {relationship: "allowed in"}]->(e2:entity {name: "carry-on"}) RETURN e1, e2
Graph Store Response:
{'e1': [], 'e2': []}
Final Response: The information available does not specify whether lithium batteries are allowed in carry-on luggage.
The Cypher Query is:



```cypher
```cypher
MATCH (e1:entity {name: "Lithium batteries"})-[:relationship {relationship: "allowed in"}]->(e2:entity {name: "carry-on"})
RETURN e1, e2
```
```


[ERROR]:
 Query Failed:
 SyntaxError: syntax error near ```cypher'
The Answer is:


<b>The information available does not specify whether lithium batteries are allowed in carry-on luggage.</b>

In [ ]:
question = "I'm flying American Airlines. Are Lithium batteries allowed in my carry-on?"

response_nl2kg = nl2kg_query_engine.query(question)

# Cypher:
print("The Cypher Query is:")
query_string = nl2kg_query_engine.generate_query(question)
display(
    Markdown(
        f"""
```cypher
{query_string}
```
"""
    )
)

%ngql {query_string}

# Answer:
print("The Answer is:")
display(Markdown(f"<b>{response_nl2kg}</b>"))

In [95]:
query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
    # response_mode="tree_summarize",
    verbose=True,
)

In [96]:
question = "I'm flying American Airlines. Are Lithium batteries allowed in my carry-on?"
response = query_engine.query(question)
display(Markdown(f"<b>{response}</b>"))

<b>Yes, Lithium batteries are allowed in carry-on bags when flying American Airlines.</b>

In [97]:
graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    llm=llm,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

In [98]:
response = query_engine_with_nl2graphquery.query("Tell me about Lithium batteries")

display(Markdown(f"<b>{response}</b>"))

<b>Lithium batteries are a type of battery that is intended for personal use and is commonly found in various devices such as laptops. They are also known to be individually protected and are often safely packaged in carry-on bags. Additionally, lithium batteries have different variations like lithium-ion, lithium polymer, and lithium iron phosphate batteries.</b>

In [99]:
response.metadata

{'f3abcb05-31cd-4fe2-a0f1-be22eab6b675': {'kg_rel_map': {'Lithium{name: Lithium}': ['Lithium{name: Lithium} <-[relationship:{relationship: Containing}]- Devices{name: Devices} -[relationship:{relationship: Are not intended for}]-> Personal use{name: Personal use}',
    'Lithium{name: Lithium} <-[relationship:{relationship: Contain}]- Cells{name: Cells} -[relationship:{relationship: Are intended for}]-> Personal use{name: Personal use}',
    'Lithium{name: Lithium} <-[relationship:{relationship: Containing}]- Devices{name: Devices} -[relationship:{relationship: Containing}]-> Cells{name: Cells}',
    'Lithium{name: Lithium} -[relationship:{relationship: Is type of}]-> Battery{name: Battery} -[relationship:{relationship: Is}]-> Protected{name: Protected}',
    'Lithium{name: Lithium} -[relationship:{relationship: Is type of}]-> Battery{name: Battery} -[relationship:{relationship: Is}]-> Lithium battery acceptance policy{name: Lithium battery acceptance policy}',
    'Lithium{name: Lithiu

In [103]:
response = query_engine_with_nl2graphquery.query("Tell me about American Airlines")

display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>American Airlines is marketed and operated by American. Itineraries are marketed and operated by American Airlines. American Airlines is also associated with Philz, which is involved in various processes like verification, resolution, reimbursement, and decision-making.</b>

In [101]:
response.metadata

{'74bb504c-a363-49c8-a950-5ea45c43ddbe': {'kg_rel_map': {'American{name: American}': ['American{name: American} <-[relationship:{relationship: Is}]- App{name: App} -[relationship:{relationship: Set up}]-> Push notifications{name: Push notifications}',
    'American{name: American} <-[relationship:{relationship: Is}]- App{name: App} <-[relationship:{relationship: On}]- Check-in{name: Check-in}',
    "American{name: American} <-[relationship:{relationship: Is}]- App{name: App} <-[relationship:{relationship: Don't have}]- App{name: App}",
    'American{name: American} <-[relationship:{relationship: Is}]- App{name: App} <-[relationship:{relationship: Open}]- American{name: American}',
    'American{name: American} -[relationship:{relationship: Marketed}]-> Itineraries{name: Itineraries} <-[relationship:{relationship: Markets and operates}]- American airlines{name: American airlines}',
    'American{name: American} -[relationship:{relationship: Marketed}]-> Itineraries{name: Itineraries} <-

In [111]:
response = query_engine_with_nl2graphquery.query("Can my carry-on be musical instruments?")

display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.


<b>Empty Response</b>

In [112]:
response = query_engine.query("Can my carry-on be musical instruments?")

display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.


<b>Empty Response</b>

In [ ]:
response = query_engine.query("Tell me about American Airlines?")
display(Markdown(f"<b>{response}</b>"))